In [ ]:
import fceulib
from fceulib import VectorBytes
import numpy as np

In [ ]:

START = 0x08
RUN = 0x02
JUMP = 0x01
mario_y = 0x00CE
mario_x = 0x006D

def hold(mask, duration):
    return [mask for i in range(duration)]


In [ ]:

emu = fceulib.runGame('mario.nes')
startInputs = hold(0x0, 120) + hold(START | JUMP, 30) + hold(0x0, 150)
jumpInputs = [hold(0x0, 5) + hold(JUMP, t) + hold(0x0, 300)
              for t in range(5, 300)]
for m in startInputs:
    emu.step(m, 0x0)
emu.stepFull(0x0, 0x0)
start = VectorBytes()
print("SAVE")
emu.save(start)

In [ ]:
def pointer_to_numpy(ptr):
    return np.array([ptr[xx] for xx in range(len(ptr))])

emu.load(start)
nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
ram_ptr = emu.fc.fceu.RAM
nt = nt[:960].reshape(30,32)

In [ ]:
import numpy as np
nt = np.array(nt)

import matplotlib.pyplot as plt
#plt.imshow(nt,interpolation='none')
#plt.show()

In [ ]:
pt = pointer_to_numpy(emu.fc.ppu.PALRAM)
print pt

In [ ]:
sprite_ram  = pointer_to_numpy(emu.fc.ppu.SPRAM)
def sprite_attributes_to_dict(attributes):
    masks = {'background' : 0x20,
        'hflip' : 0x40,
        'vflip'  : 0x80}
    sprite_attributes = {mask:attributes&masks[mask]>0 for mask in masks}
    sprite_attributes['palette'] = attributes & 0x03
    return sprite_attributes
to_plot = []
for ii in range(0,len(sprite_ram),4):
    print 'x: {}, y: {}, id:{}, attributes:{}'.format(sprite_ram[ii+3],
                                                      sprite_ram[ii],
                                                      sprite_ram[ii+1],
                                                      sprite_attributes_to_dict(sprite_ram[ii+2]))
    if sprite_ram[ii+1] > 0:
        to_plot.append([sprite_ram[ii+3],sprite_ram[ii]])
to_plot = np.array(to_plot,dtype='float')
print to_plot
plt.plot(to_plot[:,0]/8.,to_plot[:,1]/8.,'rx')
plt.imshow(nt,interpolation='none')
plt.show()


In [ ]:
help(fceulib)